# Assemble BCell Data and Assign Isotypes

PseudoCode:  

* Download CertPro B cell anndata objects (all-b-cell and memory b w/ clusters)
* Assemble into a single object with memory B clusters
* Perform Isotype assignment
     * Save cell-level table of isotype assignments for each cutoff
* Make pseudobulks by B cell L3 type and isotype
     * Save H5ad
* Check All cell types for positivity against IgH isotype thresholds
    * Save table of percent cells positive per L3 celltype per IgH

## Set Up

In [1]:
!! python --version

['Python 3.7.12']

In [2]:
import sys
sys.path.insert(0, '/home/jupyter/hisepy/build/lib')

In [3]:
import hisepy as hp

In [4]:
import anndata as ad
import scanpy as sc
import pandas as pd 
import numpy as np
import re
import matplotlib

import h5py
import statistics as stats
import os

/home/jupyter/env/ra_scrna/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import matplotlib.pyplot as plt
from itertools import combinations 

In [6]:
def plotdim(w=7,h=7):
    matplotlib.rcParams['figure.figsize'] = [w,h]

In [7]:
outdir = './_output/certpro'

In [8]:
if not os.path.exists(outdir):
    print("Creating directory {}".format(outdir))
    os.mkdir(outdir)

## Download Data

In [9]:
os.getcwd()

'/home/jupyter/ra-longitudinal/scRNA/Aim3_switched-followup'

In [10]:
fid_alldata = '609f7543-d4d5-41e9-a3d2-8e50c3e7c61d'
fid_bmem_clusters = '89859cec-8d36-47fd-a165-f0f34ae90284'  # 15.4gb
fid_in = [fid_alldata,fid_bmem_clusters]

In [13]:
hp.cache_files(file_ids = fid_in)

downloading fileID: 609f7543-d4d5-41e9-a3d2-8e50c3e7c61d
downloading fileID: 89859cec-8d36-47fd-a165-f0f34ae90284
Files have been successfully downloaded!


## Read and Format Data

### Full object

In [14]:
adata_all_raw = sc.read_h5ad("/home/jupyter/cache/609f7543-d4d5-41e9-a3d2-8e50c3e7c61d/ALTRA_certPro_scRNA_141_samples_combined_adata.h5ad")

/home/jupyter/env/ra_scrna/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [15]:
adata_all_raw.obs.columns

Index(['barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id',
       'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads',
       'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'well_id',
       'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid',
       'subject.biologicalSex', 'subject.race', 'subject.ethnicity',
       'subject.birthYear', 'sample.visitName', 'sample.drawDate',
       'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit',
       'file.id', 'subset_grp', 'predicted_doublet', 'doublet_score',
       'AIFI_L1', 'AIFI_L1_score', 'AIFI_L2', 'AIFI_L2_score', 'AIFI_L3',
       'AIFI_L3_score', 'n_genes_by_counts', 'log1p_n_genes_by_counts',
       'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes',
       'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes',
       'pct_counts_in_top_500_genes', 'total_counts_mito',
       'log1p_total_counts_mito', 'pct_counts_mito', 'leiden_harmony_2',


In [16]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(adata_all_raw.obs.loc[adata_all_raw.obs.AIFI_L1 == 'B cell',].AIFI_L3_new.value_counts())

Core naive B cell                       109207
Core memory B cell                       38623
Transitional B cell                      15358
CD27+ effector B cell                     6556
ISG+ naive B cell                         6302
CD27- effector B cell                     6221
Type 2 polarized memory B cell            3267
Plasma cell                               2050
CD95 memory B cell                        2002
Early memory B cell                       1451
Activated memory B cell                    350
Core naive CD8 T cell                       97
Early memory B cell_uk1                     83
CM CD4 T cell                               59
T2MBC_uk1                                   47
Activated memory B cell_uk1                 32
KLRF1- GZMB+ CD27- memory CD4 T cell         9
GZMB- CD27+ EM CD4 T cell                    9
Core naive CD4 T cell                        6
KLRF1- GZMB+ CD27- EM CD8 T cell             5
GZMK- CD56dim NK cell                        5
ILC          

### B Mem object

In [17]:
adata_bmem = sc.read_h5ad("/home/jupyter/cache/89859cec-8d36-47fd-a165-f0f34ae90284/ALTRA_scRNA_Bmem_cells_filtered_certPro.h5ad")

In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(adata_bmem.obs.AIFI_L3_new.value_counts())

Core memory B cell                38569
CD27+ effector B cell              6453
CD27- effector B cell              6213
Type 2 polarized memory B cell     3264
CD95 memory B cell                 1999
Early memory B cell                1448
Activated memory B cell             349
Name: AIFI_L3_new, dtype: int64


### Subset All B

In [19]:
b_types = [
    'Core naive B cell','Core memory B cell','Transitional B cell','CD27+ effector B cell',
    'ISG+ naive B cell','CD27- effector B cell','Type 2 polarized memory B cell',
    'Plasma cell','CD95 memory B cell','Early memory B cell','Activated memory B cell',
    'Activated memory B cell_uk1'
]
bmem_types = [
    'Core memory B cell','CD27+ effector B cell','CD27- effector B cell','Type 2 polarized memory B cell',
    'CD95 memory B cell','Early memory B cell','Activated memory B cell'
]
bother_types = [
    'Core naive B cell','Transitional B cell','ISG+ naive B cell','Plasma cell'
]
bnaive_types = [
    'Core naive B cell','Transitional B cell','ISG+ naive B cell'
]
all([x in adata_all_raw.obs.AIFI_L3_new.values for x in b_types])

True

In [20]:
# Bmem object counts
adata_bmem.obs.AIFI_L3_new.value_counts()

Core memory B cell                38569
CD27+ effector B cell              6453
CD27- effector B cell              6213
Type 2 polarized memory B cell     3264
CD95 memory B cell                 1999
Early memory B cell                1448
Activated memory B cell             349
Name: AIFI_L3_new, dtype: int64

In [21]:
adata_allb = adata_all_raw[adata_all_raw.obs.AIFI_L3_new.isin(b_types),].copy()

In [22]:
# All-cell object, B cell counts
adata_allb.obs.AIFI_L3_new.value_counts()

Core naive B cell                 109212
Core memory B cell                 38623
Transitional B cell                15358
CD27+ effector B cell               6556
ISG+ naive B cell                   6302
CD27- effector B cell               6221
Type 2 polarized memory B cell      3267
Plasma cell                         2053
CD95 memory B cell                  2002
Early memory B cell                 1451
Activated memory B cell              350
Activated memory B cell_uk1           32
Name: AIFI_L3_new, dtype: int64

In [23]:
# keep bmem cells only if in bmem object, keep all other types
bc_mem = adata_bmem.obs.barcodes.values.tolist()
print(len(bc_mem))
bc_other = [adata_allb.obs.barcodes.values[i] for i in range(adata_allb.obs.shape[0]) if adata_allb.obs.AIFI_L3_new[i] in bother_types]
print(len(bc_other))
bc_keep = bc_mem.copy()
bc_keep.extend(bc_other)
print(len(bc_keep))

58295
132925
191220


In [24]:
adata_allb = adata_allb[adata_allb.obs.barcodes.isin(bc_keep),].copy()

In [25]:
adata_allb

AnnData object with n_obs × n_vars = 191220 × 33538
    obs: 'barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id', 'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads', 'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'well_id', 'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid', 'subject.biologicalSex', 'subject.race', 'subject.ethnicity', 'subject.birthYear', 'sample.visitName', 'sample.drawDate', 'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit', 'file.id', 'subset_grp', 'predicted_doublet', 'doublet_score', 'AIFI_L1', 'AIFI_L1_score', 'AIFI_L2', 'AIFI_L2_score', 'AIFI_L3', 'AIFI_L3_score', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'leiden_harmony_2', 'doublets_manual', 'AIFI_L3_new', 'St

In [26]:
# All-cell object, B cell counts
adata_allb.obs.AIFI_L3_new.value_counts()

Core naive B cell                 109212
Core memory B cell                 38569
Transitional B cell                15358
CD27+ effector B cell               6453
ISG+ naive B cell                   6302
CD27- effector B cell               6213
Type 2 polarized memory B cell      3264
Plasma cell                         2053
CD95 memory B cell                  1999
Early memory B cell                 1448
Activated memory B cell              349
Name: AIFI_L3_new, dtype: int64

### All B Cell Count Check

Verify calculation method of # of b cells per sample (variable present in memory b cell obs)

In [27]:
all_b_df = adata_all_raw[adata_all_raw.obs.AIFI_L1.isin(['B cell']),].obs.copy()

In [28]:
all_b_df.groupby('sample.sampleKitGuid').size()

sample.sampleKitGuid
KT00052    1432
KT00055     880
KT00056    1136
KT00057     970
KT00058     912
           ... 
KT04905     936
KT04906    1727
KT04924    1621
KT04933    2400
KT04937     907
Length: 141, dtype: int64

In [29]:
df_b_counts = adata_bmem.obs.loc[:,['sample.sampleKitGuid','total_b_counts']].drop_duplicates().sort_values(by = 'sample.sampleKitGuid')

In [30]:
bcount_check = all_b_df.groupby('sample.sampleKitGuid').size().values == df_b_counts.total_b_counts
bcount_check.all()

True

### Add BMem Cluster Labels

In [31]:
adata_allb.obs['index'] = adata_allb.obs.index

In [32]:
adata_allb.obs.shape

(191220, 61)

In [33]:
adata_allb

AnnData object with n_obs × n_vars = 191220 × 33538
    obs: 'barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id', 'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads', 'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'well_id', 'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid', 'subject.biologicalSex', 'subject.race', 'subject.ethnicity', 'subject.birthYear', 'sample.visitName', 'sample.drawDate', 'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit', 'file.id', 'subset_grp', 'predicted_doublet', 'doublet_score', 'AIFI_L1', 'AIFI_L1_score', 'AIFI_L2', 'AIFI_L2_score', 'AIFI_L3', 'AIFI_L3_score', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'leiden_harmony_2', 'doublets_manual', 'AIFI_L3_new', 'St

In [34]:
adata_bmem.obs.columns

Index(['barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id',
       'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads',
       'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'well_id',
       'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid',
       'subject.biologicalSex', 'subject.race', 'subject.ethnicity',
       'subject.birthYear', 'sample.visitName', 'sample.drawDate',
       'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit',
       'file.id', 'subset_grp', 'predicted_doublet', 'doublet_score',
       'AIFI_L1', 'AIFI_L1_score', 'AIFI_L2', 'AIFI_L2_score', 'AIFI_L3',
       'AIFI_L3_score', 'n_genes_by_counts', 'log1p_n_genes_by_counts',
       'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes',
       'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes',
       'pct_counts_in_top_500_genes', 'total_counts_mito',
       'log1p_total_counts_mito', 'pct_counts_mito', 'leiden_harmony_2',


In [35]:
cluster_vals = adata_bmem.obs[['barcodes','leiden_0_8','leiden_0_8_anno']].copy()
cluster_vals.head()

,barcodes,leiden_0_8,leiden_0_8_anno
9291,2f3d3192226811eea184aec535c48243,4,CD27+ effector B cell
9292,b5aa7f4c120211eb859a46b4e30ed972,5,CD27+ effector B cell
9293,ef4fc95e429511eca55b3612ac783560,4,CD27+ effector B cell
9294,456f1d76428711ecbe4f8ede1e1ebd6b,1,CD27+ effector B cell
9295,f53ac52e429c11ec80811e9b3f1bfe57,4,CD27+ effector B cell


In [36]:
adata_allb.obs = adata_allb.obs.join(cluster_vals.set_index('barcodes'), on='barcodes', how='left')

In [37]:
adata_allb.obs = adata_allb.obs.join(df_b_counts.set_index('sample.sampleKitGuid'), on='sample.sampleKitGuid', how='left')

In [38]:
adata_allb.obs.shape

(191220, 64)

In [39]:
adata_allb.obs.columns

Index(['barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id',
       'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads',
       'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'well_id',
       'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid',
       'subject.biologicalSex', 'subject.race', 'subject.ethnicity',
       'subject.birthYear', 'sample.visitName', 'sample.drawDate',
       'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit',
       'file.id', 'subset_grp', 'predicted_doublet', 'doublet_score',
       'AIFI_L1', 'AIFI_L1_score', 'AIFI_L2', 'AIFI_L2_score', 'AIFI_L3',
       'AIFI_L3_score', 'n_genes_by_counts', 'log1p_n_genes_by_counts',
       'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes',
       'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes',
       'pct_counts_in_top_500_genes', 'total_counts_mito',
       'log1p_total_counts_mito', 'pct_counts_mito', 'leiden_harmony_2',


In [40]:
adata_allb.obs.leiden_0_8.value_counts()

0     9125
1     8796
2     7260
3     6379
4     6202
5     5948
6     3377
7     3353
8     3080
9     2543
10    2174
11      58
Name: leiden_0_8, dtype: int64

In [41]:
adata_allb.obs.total_b_counts.describe()

count    191220.000000
mean       1687.208284
std         788.327508
min         181.000000
25%        1176.000000
50%        1554.000000
75%        2117.000000
max        4453.000000
Name: total_b_counts, dtype: float64

In [42]:
adata_allb.obs.total_b_counts.isnull().any()

False

In [43]:
adata_allb.obs.head()

,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,BMI,CMV_Status_Subj,age_conv,bmi_conv,file.batchID,status,index,leiden_0_8,leiden_0_8_anno,total_b_counts
9291,2f3d3192226811eea184aec535c48243,B159,regulable_integral_leafwing,2f3d3192226811eea184aec535c48243,B159-P1C1,TTCCGCCTCTCTTTG,singlet,1457,170,19343,...,20.079601,NaN,NaN,NaN,B159,ALTRA_healthy,9291,4,CD27+ effector B cell,2365
9292,b5aa7f4c120211eb859a46b4e30ed972,B026,fungiform_punctual_ram,b5aa7f4c120211eb859a46b4e30ed972,B026-P2C2,TGTCTTTCCTGCCAG,singlet,2478,282,25517,...,23.699600,Negative,58.0,25.648918,B026,at_risk,9292,5,CD27+ effector B cell,1484
9293,ef4fc95e429511eca55b3612ac783560,B087,semisolemn_tyrannical_tarsier,ef4fc95e429511eca55b3612ac783560,B087-P2C2,TTCCGCCTCTCTTTG,singlet,1988,266,31449,...,26.820610,Negative,NaN,NaN,B087,at_risk,9293,4,CD27+ effector B cell,1298
9294,456f1d76428711ecbe4f8ede1e1ebd6b,B087,dubnium_weakminded_termite,456f1d76428711ecbe4f8ede1e1ebd6b,B087-P2C3,CTCCTCTGCAATTAC,singlet,1823,208,29279,...,40.538375,Positive,46.0,42.650542,B087,at_risk,9294,1,CD27+ effector B cell,1290
9295,f53ac52e429c11ec80811e9b3f1bfe57,B087,bosky_reliable_robin,f53ac52e429c11ec80811e9b3f1bfe57,B087-P2C2,TTCCGCCTCTCTTTG,singlet,1825,354,26537,...,26.820610,Negative,NaN,NaN,B087,at_risk,9295,4,CD27+ effector B cell,1298


Check a random sample to verify accuracy

In [44]:
adata_allb.obs[adata_allb.obs.barcodes == '456f1d76428711ecbe4f8ede1e1ebd6b']

,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,BMI,CMV_Status_Subj,age_conv,bmi_conv,file.batchID,status,index,leiden_0_8,leiden_0_8_anno,total_b_counts
9294,456f1d76428711ecbe4f8ede1e1ebd6b,B087,dubnium_weakminded_termite,456f1d76428711ecbe4f8ede1e1ebd6b,B087-P2C3,CTCCTCTGCAATTAC,singlet,1823,208,29279,...,40.538375,Positive,46.0,42.650542,B087,at_risk,9294,1,CD27+ effector B cell,1290


In [45]:
adata_bmem.obs[adata_bmem.obs.barcodes == '456f1d76428711ecbe4f8ede1e1ebd6b']

,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,leiden_0_5,leiden_0_8,leiden_1,leiden_1_2,swit_effect_leiden,leiden_0_8_anno
9294,456f1d76428711ecbe4f8ede1e1ebd6b,B087,dubnium_weakminded_termite,456f1d76428711ecbe4f8ede1e1ebd6b,B087-P2C3,CTCCTCTGCAATTAC,singlet,1823,208,29279,...,38.676448,0,0.0,0.0,0,1,1,1,CD27+ effector B cell,CD27+ effector B cell


### Scanpy Processing

In [46]:
# confirm X contains raw counts
adata_allb.X[1:30,1:30].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [47]:
# save raw and re-normalize
adata_allb.layers['X_raw'] = adata_allb.X.copy()
sc.pp.normalize_total(adata_allb, target_sum=10000)
adata_allb.layers['X_norm'] = adata_allb.X.copy()
sc.pp.log1p(adata_allb)
adata_allb.layers['X_norm_log1p'] = adata_allb.X.copy()
sc.pp.scale(adata_allb)
adata_allb.layers['X_scale'] = adata_allb.X.copy()

## Variables

In [48]:
ig_genes = ['IGHG1','IGHG2','IGHG3','IGHG4','IGHA1','IGHA2','IGHM','IGHD','IGHE']

## All Cell Isotype Threshold Proportions

Check IGH UMI countsfor all cell types (including non-B cells). 
* Apply same thresholds to see what percent of cells per celltype are 'positive' for IGH
* Output table for downstream visualization

In [49]:
temp_cols = adata_all_raw.obs.columns.tolist()
temp_cols.extend(ig_genes)

In [50]:
adata_all_raw.layers['X_raw'] = adata_all_raw.X.copy()
sc.pp.normalize_total(adata_all_raw, target_sum=10000)
adata_all_raw.layers['X_norm'] = adata_all_raw.X.copy()

In [51]:
df_all_cells = sc.get.obs_df(adata_all_raw, keys = temp_cols, layer='X_norm')

In [52]:
def pct_gt_cutoff(cutoff):
    def pct_gt_cutoff_(x):
        temp = x>cutoff
        ngt = sum(temp)
        ntot = len(x)
        pct_gt = ngt/ntot*100
        return pct_gt
    pct_gt_cutoff_.__name__ = 'pct_gt_cutoff_{}'.format(cutoff)
    return pct_gt_cutoff_

In [54]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:02.0f}'.format(n*100)
    return percentile_

In [55]:
fun_names =  [pct_gt_cutoff(0.5), pct_gt_cutoff(2), 'size']
fun_dict = {x: fun_names for x in ig_genes}
all_cell_iso_prop = df_all_cells.groupby(['AIFI_L3_new']).agg(fun_dict)

In [56]:
type(all_cell_iso_prop)

pandas.core.frame.DataFrame

In [57]:
all_cell_iso_prop.columns = [' '.join(col).strip() for col in all_cell_iso_prop.columns.values]

In [58]:
all_cell_iso_prop.columns

Index(['IGHG1 pct_gt_cutoff_0.5', 'IGHG1 pct_gt_cutoff_2', 'IGHG1 size',
       'IGHG2 pct_gt_cutoff_0.5', 'IGHG2 pct_gt_cutoff_2', 'IGHG2 size',
       'IGHG3 pct_gt_cutoff_0.5', 'IGHG3 pct_gt_cutoff_2', 'IGHG3 size',
       'IGHG4 pct_gt_cutoff_0.5', 'IGHG4 pct_gt_cutoff_2', 'IGHG4 size',
       'IGHA1 pct_gt_cutoff_0.5', 'IGHA1 pct_gt_cutoff_2', 'IGHA1 size',
       'IGHA2 pct_gt_cutoff_0.5', 'IGHA2 pct_gt_cutoff_2', 'IGHA2 size',
       'IGHM pct_gt_cutoff_0.5', 'IGHM pct_gt_cutoff_2', 'IGHM size',
       'IGHD pct_gt_cutoff_0.5', 'IGHD pct_gt_cutoff_2', 'IGHD size',
       'IGHE pct_gt_cutoff_0.5', 'IGHE pct_gt_cutoff_2', 'IGHE size'],
      dtype='object')

In [59]:
all_cell_iso_prop.head()

,IGHG1 pct_gt_cutoff_0.5,IGHG1 pct_gt_cutoff_2,IGHG1 size,IGHG2 pct_gt_cutoff_0.5,IGHG2 pct_gt_cutoff_2,IGHG2 size,IGHG3 pct_gt_cutoff_0.5,IGHG3 pct_gt_cutoff_2,IGHG3 size,IGHG4 pct_gt_cutoff_0.5,...,IGHA2 size,IGHM pct_gt_cutoff_0.5,IGHM pct_gt_cutoff_2,IGHM size,IGHD pct_gt_cutoff_0.5,IGHD pct_gt_cutoff_2,IGHD size,IGHE pct_gt_cutoff_0.5,IGHE pct_gt_cutoff_2,IGHE size
AIFI_L3_new,,,,,,,,,,,,,,,,,,,,,
ASDC,0.566572,0.000000,353,0.000000,0.000000,353,0.000000,0.000000,353,0.283286,...,353,32.294618,2.832861,353,14.447592,0.283286,353,0.000000,0.000000,353
ASDC_uk1_B,0.000000,0.000000,164,0.000000,0.000000,164,0.000000,0.000000,164,0.000000,...,164,68.292683,26.829268,164,19.512195,0.000000,164,0.000000,0.000000,164
Activated memory B cell,28.000000,13.714286,350,26.571429,12.571429,350,25.714286,13.142857,350,8.857143,...,350,78.285714,76.285714,350,54.000000,45.428571,350,2.285714,1.714286,350
Activated memory B cell_uk1,25.000000,15.625000,32,40.625000,15.625000,32,65.625000,46.875000,32,18.750000,...,32,100.000000,100.000000,32,84.375000,71.875000,32,0.000000,0.000000,32
Adaptive NK cell,0.275957,0.213867,14495,0.131080,0.124181,14495,0.296654,0.248362,14495,0.262159,...,14495,2.449120,1.848913,14495,0.310452,0.255260,14495,0.034495,0.027596,14495


### Save

Save out table for plotting and further summary

In [60]:
all_cell_iso_prop.to_csv("./_output/certpro/check_igh_normcounts_alltypes.csv")

## Isotype Classification

### Functions

In [61]:
# 1) https://stackoverflow.com/questions/26784164/pandas-multiprocessing-apply
# 2) https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments
from multiprocessing import Pool
from itertools import repeat
from functools import partial

# 3) https://stackoverflow.com/questions/817087/call-a-function-with-argument-list-in-python
def mywrapper(func, args): 
    return func(*args)

def parallelize_row_args(df, df_func, args=[], num_of_processes=8):
    ''' Adapted code found in resource #1 with starmap for arguments (resource #2)
    
    Params
    ------
    df pandas.DataFrame
        A dataframe object we wish to parallelize row operations on
    df_func function
        A function that takes a dataframe as the first argument

    '''
    # Set up processes and split data
    data_split = np.array_split(df, num_of_processes)
    pool = Pool(num_of_processes)
    
    # Run fun on each split
    if args is None or len(args) == 0:
        data = pd.concat(pool.map(df_func, data_split))
    else:
        # Set up argument list for each split process if additional args
        arglist = [data_split]
        arglist.extend([repeat(x) for x in args])
        data = pd.concat(pool.starmap(df_func, mywrapper(zip, arglist)))
    
    # Shut down pool
    pool.close()
    pool.join()
    
    return data


In [62]:
def count_pos(df, ig_genes = ig_genes):
    res = df.apply(lambda x: sum(x[ig_genes]), axis = 1)
    return res

def concat_pos(df, ig_genes = ig_genes):
    res = df.apply(lambda x: ";".join([g for g in ig_genes if x[g] is True]), axis = 1)
    return res

def get_is_ig(df, ig, other_ig):
    res = df.apply(lambda x: x[ig] and not any(x[other_ig]), axis = 1)
    return res

def get_is_nonswitched(df, nonswitched, switched):
    res = df.apply(lambda x: any(x[nonswitched]) and not any(x[switched]), axis = 1)
    return res

def determine_isotype(df, gene_order_arr, pos_str_col = 'concat_pos', verbose = True):
    x = df.copy()
    res = ["undetermined" for x in range(df.shape[0])]
    
    for gene in gene_order_arr:
        if not isinstance(gene, str):
            bool_arr = [pd.notna(df[pos_str_col].str.extract('('+g+')')[0]) for g in gene]
            i = 1
            pat_found = bool_arr[0]
            while i < len(bool_arr):
                pat_found = np.logical_and(pat_found, bool_arr[i])
                i+=1
            gene = ";".join(gene)
        else:
            pat_found = pd.notna(x[pos_str_col].str.extract('('+gene+')')[0])
        
        pat_found = np.logical_and(pat_found, [x == 'undetermined' for x in res])
        n_found = len([x for x in pat_found if x is True])
        if verbose:
            print("{} matches for gene {}".format(n_found, gene))
        
        if n_found > 0:
            ifound = [i for i in range(len(pat_found)) if pat_found[i] == True]
            for ichange in ifound:
                res[ichange] = gene
    return res
            

def classify_ig_df(
    df,
    key = 'barcodes',
    ig_cutoffs = {
       'IGHG1':1,
       'IGHG2':1,
       'IGHG3':1,
       'IGHG4':1,
       'IGHA1':1,
       'IGHA2':1,
       'IGHM':1,
       'IGHD':1,
       'IGHE':1
    },
    switched = ['IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHA1', 'IGHA2','IGHE'],
    nonswitched = ['IGHM', 'IGHD'],
    gene_order_arr = [["IGHM","IGHD"], "IGHM","IGHD","IGHG3","IGHG1","IGHA1","IGHG2","IGHG4","IGHE","IGHA2"],
    keep_normcounts = True,
    num_of_processes = 8,
    verbose = True
):
    
    ''' Classify Ig status for wide dataframe of Ig Norm Counts

    Cells are labeled with switched isotypes a single switched isotype is above (gte) threshold and all other 
    isotypes are less than threshold. Cells are labeled with nonswitched isotypes when all switched isotypes are
    below threshold and one or more nonswitched isotype is above threshold.
    
    Algorithm developed with input from Ziyuan He and Marla Glass. Thresholds may be dataset specific but
    defaults were applied in normalized counts from a 3' scRNA dataset (ALTRA)

    Params
    ------
    df: pandas DataFrame of normalized ig counts
        Dataframe with rows as cells and columns as featres, eg as extracted from an AnnData object 
        using sc.get.obs_df(). Must contain a key column (e.g. barcodes) in addition to columns for
        all ig_genes.
    key: str, default 'barcodes'
        Column in df that is the unique row key
    ig_cutoffs: dict
        Dictionary with names as ig genes and values as cutoff values in the same units (eg normalized counts) 
        as Ig values in df. 
    switched: list, default ['IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHA1', 'IGHA2','IGHE'],
    nonswitched: list, default ['IGHM', 'IGHD'],
    keep_normcounts: bool, default True
        If True the original counts in df will be retained in output. 
    verbose: bool, default True
    
    
    '''
    ig_genes = switched.copy()
    ig_genes.extend(nonswitched)
    
    missing_keys = [x for x in ig_genes if x not in ig_cutoffs.keys()]
    if len(missing_keys) > 0:
        raise KeyError("Missing ig cuttoffs for the following: {}".format(", ".join(missing_keys)))
    
    if key not in df.columns:
        raise KeyError("Missing key column '{}' in data frame".format(key))
                                                                           
    if verbose:
        print("Binarizing Ig detection based on cutoffs")
    bool_df = pd.DataFrame({x: df[x]>= ig_cutoffs[x] for x in ig_genes})
    newcols = [x+"_pos" for x in ig_genes]
    bool_df['n_pos'] =  parallelize_row_args(bool_df, count_pos, args = [ig_genes], num_of_processes= num_of_processes)
    bool_df['concat_pos'] =  parallelize_row_args(bool_df, concat_pos, args = [ig_genes], num_of_processes= num_of_processes)
    bool_df['switched_status'] = 'undetermined'
    
    # Determine switch status
    if verbose:
        print("Determining switched status")
    # array of logical subarrays. one subarray per nonswitched gene. bool values for whether cell was pos for gene
    bool_arr = [pd.notna(bool_df.concat_pos.str.extract('('+g+')')[0]) for g in nonswitched]
    is_nonswitched = bool_arr[0]
    i = 1
    while i < len(bool_arr):
        is_nonswitched = np.logical_or(is_nonswitched, bool_arr[i])
        i+=1
    i_nonswitched = bool_df.index[is_nonswitched]
    bool_df.loc[i_nonswitched, 'switched_status'] = 'nonswitched'
    is_switched = np.logical_and(np.logical_not(is_nonswitched), bool_df.concat_pos != '')
    iswitched = bool_df.index[is_switched]
    bool_df.loc[iswitched, 'switched_status'] = 'switched'

    
    # predict isotype
    if verbose:
        print("Predicting isotypes")
    bool_df['isotype'] = determine_isotype(bool_df, gene_order_arr = gene_order_arr, pos_str_col = 'concat_pos', verbose = True)

    # Format for output
    bool_df.rename(columns={ig_genes[i]: newcols[i] for i in range(len(ig_genes))}, inplace=True)

    keep_cols = [key]
    if keep_normcounts is True:
        keep_cols.extend(ig_genes)
        
    res = pd.concat([df[keep_cols], bool_df], axis=1)
        
    return res

### Determine Class Switch and Isotype

In [63]:
# extract key marker data to flat dataframe
keepcols = adata_allb.obs.columns.tolist()
keepcols.extend(ig_genes)
df = sc.get.obs_df(adata_allb, keys = keepcols, layer='X_norm')
print(df.shape)
df.head()

(191220, 73)


,barcodes,batch_id,cell_name,cell_uuid,chip_id,hto_barcode,hto_category,n_genes,n_mito_umis,n_reads,...,total_b_counts,IGHG1,IGHG2,IGHG3,IGHG4,IGHA1,IGHA2,IGHM,IGHD,IGHE
9291,2f3d3192226811eea184aec535c48243,B159,regulable_integral_leafwing,2f3d3192226811eea184aec535c48243,B159-P1C1,TTCCGCCTCTCTTTG,singlet,1457,170,19343,...,2365,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,56.094929,2.157497,0.0
9292,b5aa7f4c120211eb859a46b4e30ed972,B026,fungiform_punctual_ram,b5aa7f4c120211eb859a46b4e30ed972,B026-P2C2,TGTCTTTCCTGCCAG,singlet,2478,282,25517,...,1484,1.184975,3.554924,1.184975,0.0,0.000000,0.000000,0.000000,0.000000,0.0
9293,ef4fc95e429511eca55b3612ac783560,B087,semisolemn_tyrannical_tarsier,ef4fc95e429511eca55b3612ac783560,B087-P2C2,TTCCGCCTCTCTTTG,singlet,1988,266,31449,...,1298,0.000000,0.000000,0.000000,0.0,1.372307,1.372307,27.446136,0.000000,0.0
9294,456f1d76428711ecbe4f8ede1e1ebd6b,B087,dubnium_weakminded_termite,456f1d76428711ecbe4f8ede1e1ebd6b,B087-P2C3,CTCCTCTGCAATTAC,singlet,1823,208,29279,...,1290,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,40.580616,14.047136,0.0
9295,f53ac52e429c11ec80811e9b3f1bfe57,B087,bosky_reliable_robin,f53ac52e429c11ec80811e9b3f1bfe57,B087-P2C2,TTCCGCCTCTCTTTG,singlet,1825,354,26537,...,1298,3.183699,0.000000,0.000000,0.0,1.591850,0.000000,17.510347,6.367399,0.0


In [64]:
pd.set_option('display.max_columns', None)
df.groupby(['AIFI_L3_new']).describe()

n_genes                                   \
                                   count         mean         std     min   
AIFI_L3_new                                                                 
Activated memory B cell            349.0  1776.936963  530.823429   623.0   
CD27+ effector B cell             6453.0  1981.489695  572.355466   489.0   
CD27- effector B cell             6213.0  1617.788830  468.226358   332.0   
CD95 memory B cell                1999.0  2677.537269  648.137992  1013.0   
Core memory B cell               38569.0  1578.945656  416.045756   257.0   
Core naive B cell               109212.0  1325.340494  333.421511   281.0   
Early memory B cell               1448.0  1584.896409  413.853207   542.0   
ISG+ naive B cell                 6302.0  1456.173596  434.200748   455.0   
Plasma cell                       2053.0  3226.519727  899.615559   267.0   
Transitional B cell              15358.0  1480.446543  386.317651   368.0   
Type 2 polarized memory B cell    3264.0  1533.575674  340.259560   603.0   

                                                                n_mito_umis  \
                                    25%     50%     75%     max       count   
AIFI_L3_new                                                                   
Activated memory B cell         1431.00  1733.0  2009.0  3766.0       349.0   
CD27+ effector B cell           1590.00  1912.0  2297.0  4914.0      6453.0   
CD27- effector B cell           1304.00  1558.0  1867.0  4499.0      6213.0   
CD95 memory B cell              2226.50  2578.0  3041.5  4896.0      1999.0   
Core memory B cell              1313.00  1563.0  1825.0  4472.0     38569.0   
Core naive B cell               1107.00  1299.0  1508.0  4713.0    109212.0   
Early memory B cell             1315.25  1527.0  1791.5  3951.0      1448.0   
ISG+ naive B cell               1166.00  1386.0  1656.0  4562.0      6302.0   
Plasma cell                     2634.00  3265.0  3869.0  4996.0      2053.0   
Transitional B cell             1218.00  1434.0  1689.0  4611.0     15358.0   
Type 2 polarized memory B cell  1305.75  1515.0  1733.0  3431.0      3264.0   

                                                                            \
                                      mean         std   min    25%    50%   
AIFI_L3_new                                                                  
Activated memory B cell         256.257880  141.180703  18.0  152.0  233.0   
CD27+ effector B cell           327.312568  185.140315   0.0  199.0  288.0   
CD27- effector B cell           211.776276  122.707417   1.0  127.0  186.0   
CD95 memory B cell              388.026513  246.789067   7.0  234.0  328.0   
Core memory B cell              218.243382  120.817641   0.0  135.0  195.0   
Core naive B cell               169.621534   89.357939   0.0  109.0  153.0   
Early memory B cell             222.494475  123.096616  32.0  141.0  195.0   
ISG+ naive B cell               190.784830  111.978396   1.0  118.0  167.0   
Plasma cell                     642.658548  441.293018   0.0  322.0  536.0   
Transitional B cell             175.191692   95.071839   1.0  110.0  156.0   
Type 2 polarized memory B cell  183.738664   93.467687  15.0  120.0  166.0   

                                                 n_reads                \
                                   75%     max     count          mean   
AIFI_L3_new                                                              
Activated memory B cell         324.00   799.0     349.0  21207.765043   
CD27+ effector B cell           415.00  1896.0    6453.0  25921.347900   
CD27- effector B cell           266.00  1269.0    6213.0  17546.799131   
CD95 memory B cell              469.50  2294.0    1999.0  35899.770885   
Core memory B cell              274.00  1762.0   38569.0  19207.525707   
Core naive B cell               211.00  1635.0  109212.0  14138.701654   
Early memory B cell             274.25  1264.0    1448.0  17730.479282   
ISG+ naive B c

Predict isotypes using cutoff of 2

In [65]:
%%time
ig_types2 = classify_ig_df(
    df, 
    key = 'barcodes',
    ig_cutoffs = {
       'IGHG1':2,
       'IGHG2':2,
       'IGHG3':2,
       'IGHG4':2,
       'IGHA1':2,
       'IGHA2':2,
       'IGHM':2,
       'IGHD':2,
       'IGHE':2
    },
    num_of_processes = 14
)

Binarizing Ig detection based on cutoffs
Determining switched status
Predicting isotypes
141691 matches for gene IGHM;IGHD
24856 matches for gene IGHM
513 matches for gene IGHD
6836 matches for gene IGHG3
4876 matches for gene IGHG1
8542 matches for gene IGHA1
1054 matches for gene IGHG2
627 matches for gene IGHG4
127 matches for gene IGHE
354 matches for gene IGHA2
CPU times: user 7.91 s, sys: 8.32 s, total: 16.2 s
Wall time: 21.2 s


Predict isotypes using cutoff of 0.5

In [66]:
%%time
ig_types05 = classify_ig_df(
    df, 
    key = 'barcodes',
    ig_cutoffs = {
       'IGHG1':0.5,
       'IGHG2':0.5,
       'IGHG3':0.5,
       'IGHG4':0.5,
       'IGHA1':0.5,
       'IGHA2':0.5,
       'IGHM':0.5,
       'IGHD':0.5,
       'IGHE':0.5
    },
    num_of_processes = 14
)

Binarizing Ig detection based on cutoffs
Determining switched status
Predicting isotypes
146917 matches for gene IGHM;IGHD
21357 matches for gene IGHM
607 matches for gene IGHD
9053 matches for gene IGHG3
4349 matches for gene IGHG1
7040 matches for gene IGHA1
705 matches for gene IGHG2
382 matches for gene IGHG4
59 matches for gene IGHE
203 matches for gene IGHA2
CPU times: user 8.04 s, sys: 8.23 s, total: 16.3 s
Wall time: 21.4 s


### Check Results

In [69]:
ig_types05.switched_status.value_counts()

nonswitched     168881
switched         21791
undetermined       548
Name: switched_status, dtype: int64

In [70]:
ig_types2.switched_status.value_counts()

nonswitched     167060
switched         22416
undetermined      1744
Name: switched_status, dtype: int64

In [71]:
dfbmem = df.loc[df.AIFI_L3_new.isin(bmem_types),].copy()

In [72]:
dfoth = df.loc[df.AIFI_L3_new.isin(bother_types),].copy()

In [73]:
dfnaive = df.loc[df.AIFI_L3_new.isin(bnaive_types),].copy()
dfnaive.shape

(130872, 73)

In [74]:
dfplasma = df.loc[df.AIFI_L3_new.isin(['Plasma cell']),].copy()
dfplasma.shape

(2053, 73)

In [75]:
test2 = ig_types2.loc[ig_types2.barcodes.isin(dfbmem.barcodes.values),].copy()
test2.switched_status.value_counts()

nonswitched     36201
switched        20452
undetermined     1642
Name: switched_status, dtype: int64

In [76]:
test05 = ig_types05.loc[ig_types05.barcodes.isin(dfbmem.barcodes.values),].copy()
test05.switched_status.value_counts()

nonswitched     37921
switched        19891
undetermined      483
Name: switched_status, dtype: int64

In [77]:
test05.isotype.value_counts()

IGHM;IGHD       22423
IGHM            15023
IGHG3            8176
IGHA1            6272
IGHG1            4148
IGHG2             674
undetermined      483
IGHD              475
IGHG4             365
IGHA2             200
IGHE               56
Name: isotype, dtype: int64

In [78]:
test2.isotype.value_counts()

IGHM;IGHD       18181
IGHM            17640
IGHA1            7586
IGHG3            6120
IGHG1            4663
undetermined     1642
IGHG2            1018
IGHG4             595
IGHD              380
IGHA2             347
IGHE              123
Name: isotype, dtype: int64

In [79]:
test2naive = ig_types2.loc[ig_types2.barcodes.isin(dfnaive.barcodes.values),].copy()
test2naive.switched_status.value_counts()

nonswitched     130273
switched           498
undetermined       101
Name: switched_status, dtype: int64

In [80]:
test2naive = ig_types2.loc[ig_types2.barcodes.isin(dfnaive.barcodes.values),].copy()
test2naive.switched_status.value_counts()/test2naive.shape[0]*100

nonswitched     99.542301
switched         0.380524
undetermined     0.077175
Name: switched_status, dtype: float64

In [81]:
test05naive = ig_types05.loc[ig_types05.barcodes.isin(dfnaive.barcodes.values),].copy()
test05naive.switched_status.value_counts()

nonswitched     130320
switched           487
undetermined        65
Name: switched_status, dtype: int64

In [82]:
test05naive = ig_types05.loc[ig_types05.barcodes.isin(dfnaive.barcodes.values),].copy()
test05naive.switched_status.value_counts()/test05naive.shape[0]*100

nonswitched     99.578214
switched         0.372119
undetermined     0.049667
Name: switched_status, dtype: float64

In [83]:
test2plasma = ig_types2.loc[ig_types2.barcodes.isin(dfplasma.barcodes.values),].copy()
test2plasma.switched_status.value_counts()

switched        1466
nonswitched      586
undetermined       1
Name: switched_status, dtype: int64

In [84]:
test2plasma = ig_types2.loc[ig_types2.barcodes.isin(dfplasma.barcodes.values),].copy()
test2plasma.switched_status.value_counts()/test2plasma.shape[0]*100

switched        71.407696
nonswitched     28.543595
undetermined     0.048709
Name: switched_status, dtype: float64

In [85]:
test05plasma = ig_types05.loc[ig_types05.barcodes.isin(dfplasma.barcodes.values),].copy()
test05plasma.switched_status.value_counts()

switched       1413
nonswitched     640
Name: switched_status, dtype: int64

In [86]:
test05plasma = ig_types05.loc[ig_types05.barcodes.isin(dfplasma.barcodes.values),].copy()
test05plasma.switched_status.value_counts()/test05plasma.shape[0]*100

switched       68.826108
nonswitched    31.173892
Name: switched_status, dtype: float64

In [89]:
test05oth = ig_types05.loc[ig_types05.barcodes.isin(dfoth.barcodes.values),].copy()
test05oth.switched_status.value_counts()/test05oth.shape[0]*100

nonswitched     98.521723
switched         1.429377
undetermined     0.048900
Name: switched_status, dtype: float64

In [91]:
test2oth = ig_types2.loc[ig_types2.barcodes.isin(dfoth.barcodes.values),].copy()
test2oth.switched_status.value_counts()/test2oth.shape[0]*100

nonswitched     98.445740
switched         1.477525
undetermined     0.076735
Name: switched_status, dtype: float64

In [90]:
test05oth.isotype.value_counts()

IGHM;IGHD       124494
IGHM              6334
IGHG3              877
IGHA1              768
IGHG1              201
IGHD               132
undetermined        65
IGHG2               31
IGHG4               17
IGHA2                3
IGHE                 3
Name: isotype, dtype: int64

In [92]:
test2oth.isotype.value_counts()

IGHM;IGHD       123510
IGHM              7216
IGHA1              956
IGHG3              716
IGHG1              213
IGHD               133
undetermined       102
IGHG2               36
IGHG4               32
IGHA2                7
IGHE                 4
Name: isotype, dtype: int64

In [93]:
test05.groupby(['switched_status','n_pos']).size()

switched_status  n_pos
nonswitched      1         1916
                 2         8490
                 3        11664
                 4         9228
                 5         4556
                 6         1730
                 7          309
                 8           28
switched         1         2923
                 2         7911
                 3         5426
                 4         2695
                 5          815
                 6          118
                 7            3
undetermined     0          483
dtype: int64

In [94]:
test2.groupby(['switched_status','n_pos']).size()

switched_status  n_pos
nonswitched      1         5681
                 2        13626
                 3        10210
                 4         4788
                 5         1476
                 6          373
                 7           45
                 8            2
switched         1         6548
                 2         9241
                 3         3410
                 4         1013
                 5          219
                 6           20
                 7            1
undetermined     0         1642
dtype: int64

### Save

Merge cell metadata and Ig count data to isotype results

In [67]:
df.to_csv("_output/certpro/normcount_obs_df_allb_forcheck.csv")  

In [95]:
ig_types2.columns

Index(['barcodes', 'IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHA1', 'IGHA2',
       'IGHE', 'IGHM', 'IGHD', 'IGHG1_pos', 'IGHG2_pos', 'IGHG3_pos',
       'IGHG4_pos', 'IGHA1_pos', 'IGHA2_pos', 'IGHE_pos', 'IGHM_pos',
       'IGHD_pos', 'n_pos', 'concat_pos', 'switched_status', 'isotype'],
      dtype='object')

In [96]:
df_ig = df.merge(ig_types2, how = 'left')

In [97]:
df_ig.to_csv(os.path.join(outdir, 'adata-all-b_isotypes_cutoff2.csv'), index=False)  

In [98]:
df_ig05 = df.merge(ig_types05, how = 'left')

In [99]:
df_ig05.to_csv(os.path.join(outdir, 'adata-all-b_isotypes_cutoff0.5.csv'), index=False)  

## Make Pseudobulk

Pseudobulk B cells by L3 celltype and isotype (use 0.5 cutoff for isotype assignment)

In [100]:
import decoupler as dc

In [101]:
adata_allb.obs = adata_allb.obs.merge(how='left', right=ig_types05, left_index=True, right_index=True)

In [102]:
ig_types05.columns

Index(['barcodes', 'IGHG1', 'IGHG2', 'IGHG3', 'IGHG4', 'IGHA1', 'IGHA2',
       'IGHE', 'IGHM', 'IGHD', 'IGHG1_pos', 'IGHG2_pos', 'IGHG3_pos',
       'IGHG4_pos', 'IGHA1_pos', 'IGHA2_pos', 'IGHE_pos', 'IGHM_pos',
       'IGHD_pos', 'n_pos', 'concat_pos', 'switched_status', 'isotype'],
      dtype='object')

In [103]:
for x in ig_genes:
    print(x)
    adata_allb.obs = adata_allb.obs.drop(columns=x)

IGHG1
IGHG2
IGHG3
IGHG4
IGHA1
IGHA2
IGHM
IGHD
IGHE


In [104]:
adata_allb.obs['celltype_isotype'] = [str(x) + "_" + str(y) for x,y in zip(adata_allb.obs['AIFI_L3_new'].values, adata_allb.obs['isotype'].values)]

In [105]:
adata_allb.obs.celltype_isotype.value_counts()

Core naive B cell_IGHM;IGHD      104055
Transitional B cell_IGHM;IGHD     15030
Core memory B cell_IGHM;IGHD      12662
Core memory B cell_IGHM           11058
ISG+ naive B cell_IGHM;IGHD        5375
                                  ...  
Transitional B cell_IGHG3             1
Early memory B cell_IGHG4             1
CD27- effector B cell_IGHE            1
Plasma cell_IGHE                      1
Plasma cell_IGHA2                     1
Name: celltype_isotype, Length: 104, dtype: int64

In [106]:
ig_index = [i for i in range(len(adata_allb.var_names)) if adata_allb.var_names[i] in ig_genes]

In [107]:
# use decoupler to make pseudobulk of all b cells by isotype and celltype. 
# isotype cutoff = 0.5, min cells = 20
adata_pb = dc.get_pseudobulk(
    adata = adata_allb, 
    sample_col = "sample.sampleKitGuid", 
    groups_col = 'celltype_isotype', 
    layer = 'X_raw',
    min_cells=20, 
    min_counts=1000,
    mode='sum'
)

In [108]:
# Summarize total counts in each pseudobulk
adata_pb.obs.psbulk_counts.describe()

count    1.156000e+03
mean     6.723391e+05
std      1.135757e+06
min      5.055600e+04
25%      1.618782e+05
50%      2.736015e+05
75%      5.567242e+05
max      1.113675e+07
Name: psbulk_counts, dtype: float64

In [109]:
# Summarize total counts in each pseudobulk
adata_pb.obs.psbulk_counts.describe()

count    1.156000e+03
mean     6.723391e+05
std      1.135757e+06
min      5.055600e+04
25%      1.618782e+05
50%      2.736015e+05
75%      5.567242e+05
max      1.113675e+07
Name: psbulk_counts, dtype: float64

In [110]:
[x for x in adata_pb.obs.columns if adata_pb.obs[x].dtype == 'object']

['batch_id',
 'hto_barcode',
 'hto_category',
 'pbmc_sample_id',
 'pool_id',
 'sample.sampleKitGuid',
 'cohort.cohortGuid',
 'subject.subjectGuid',
 'subject.biologicalSex',
 'subject.race',
 'subject.ethnicity',
 'subject.birthYear',
 'sample.visitName',
 'sample.drawDate',
 'sample.diseaseStatesRecordedAtVisit',
 'sample.daysSinceFirstVisit',
 'file.id',
 'subset_grp',
 'predicted_doublet',
 'AIFI_L3',
 'doublets_manual',
 'AIFI_L3_new',
 'Status_Xsec',
 'Status_Long',
 'anti_ccp3_finalCombined',
 'days_to_conversion',
 'BMI',
 'CMV_Status_Subj',
 'age_conv',
 'bmi_conv',
 'file.batchID',
 'status',
 'total_b_counts',
 'IGHM_pos',
 'IGHD_pos',
 'switched_status',
 'isotype',
 'celltype_isotype']

In [111]:
adata_pb.obs.columns

Index(['batch_id', 'hto_barcode', 'hto_category', 'pbmc_sample_id', 'pool_id',
       'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid',
       'subject.biologicalSex', 'subject.race', 'subject.ethnicity',
       'subject.birthYear', 'sample.visitName', 'sample.drawDate',
       'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit',
       'file.id', 'subset_grp', 'predicted_doublet', 'AIFI_L3',
       'doublets_manual', 'AIFI_L3_new', 'Status_Xsec', 'Status_Long',
       'anti_ccp3_finalCombined', 'days_to_conversion', 'BMI',
       'CMV_Status_Subj', 'age_conv', 'bmi_conv', 'file.batchID', 'status',
       'total_b_counts', 'IGHM_pos', 'IGHD_pos', 'switched_status', 'isotype',
       'celltype_isotype', 'psbulk_n_cells', 'psbulk_counts'],
      dtype='object')

In [112]:
adata_pb.obs.IGHM_pos

KT00057_CD27+ effector B cell_IGHM               True
KT00077_CD27+ effector B cell_IGHM               True
KT00099_CD27+ effector B cell_IGHM               True
KT00120_CD27+ effector B cell_IGHM               True
KT00221_CD27+ effector B cell_IGHM               True
                                                ...  
KT02817_Type 2 polarized memory B cell_IGHG3    False
KT02845_Type 2 polarized memory B cell_IGHG3    False
KT00069_Type 2 polarized memory B cell_IGHM      True
KT00092_Type 2 polarized memory B cell_IGHM      True
KT00477_Type 2 polarized memory B cell_IGHM      True
Name: IGHM_pos, Length: 1156, dtype: object

In [113]:
adata_pb.layers['counts'] = adata_pb.X.copy()

In [114]:
adata_pb.obs.age_conv = adata_pb.obs.age_conv.astype('float64')

In [115]:
adata_pb.obs['IGHM_pos'] = adata_pb.obs['IGHM_pos'].astype('bool')

In [116]:
adata_pb.obs['IGHD_pos'] = adata_pb.obs['IGHD_pos'].astype('bool')

In [117]:
adata_pb.obs['predicted_doublet'] = adata_pb.obs['predicted_doublet'].astype('bool')

In [118]:
adata_pb.obs['sample.daysSinceFirstVisit'] = adata_pb.obs['sample.daysSinceFirstVisit'].astype('int64')

In [119]:
adata_pb.obs.total_b_counts = adata_pb.obs.total_b_counts.astype('int64')

In [120]:
adata_pb.obs.anti_ccp3_finalCombined = adata_pb.obs.anti_ccp3_finalCombined.astype('float64')

In [121]:
adata_pb.obs['days_to_conversion'] = adata_pb.obs['days_to_conversion'].astype('float64')

In [122]:
adata_pb.obs['BMI'] = adata_pb.obs['BMI'].astype('float64')

In [123]:
adata_pb.obs['bmi_conv'] = adata_pb.obs['bmi_conv'].astype('float64')

In [124]:
adata_pb.obs['subject.birthYear'] = adata_pb.obs['subject.birthYear'].astype('int64')

In [125]:
adata_pb.obs['subject.birthYear'] = adata_pb.obs['subject.birthYear'].astype('int64')

### Save

In [126]:
out_ad_fp = os.path.join(outdir,'ALTRA_certPro_scRNA_141_samples_combined_adata_B_isotype_pseudobulk_20cell.h5ad')
adata_pb.write_h5ad(out_ad_fp)

## Make Test Pseudobulk

Pseudobulk B cells by L3 celltype only for comparison to Zi's analysis

In [127]:
adata_allb.obs.AIFI_L3_new.value_counts()

Core naive B cell                 109212
Core memory B cell                 38569
Transitional B cell                15358
CD27+ effector B cell               6453
ISG+ naive B cell                   6302
CD27- effector B cell               6213
Type 2 polarized memory B cell      3264
Plasma cell                         2053
CD95 memory B cell                  1999
Early memory B cell                 1448
Activated memory B cell              349
Name: AIFI_L3_new, dtype: int64

In [128]:
ig_index = [i for i in range(len(adata_allb.var_names)) if adata_allb.var_names[i] in ig_genes]

In [129]:
# use decoupler to make pseudobulk of all b cells by isotype and celltype. 
# isotype cutoff = 0.5, min cells = 20
adata_pb_L3 = dc.get_pseudobulk(
    adata = adata_allb, 
    sample_col = "sample.sampleKitGuid", 
    groups_col = 'AIFI_L3_new', 
    layer = 'X_raw',
    min_cells=20, 
    min_counts=1000,
    mode='sum'
)

In [130]:
# Summarize total counts in each pseudobulk
adata_pb_L3.obs.psbulk_counts.describe()

count    8.130000e+02
mean     1.064851e+06
std      1.407353e+06
min      6.040000e+04
25%      2.073790e+05
50%      4.691180e+05
75%      1.422112e+06
max      1.151130e+07
Name: psbulk_counts, dtype: float64

In [131]:
[x for x in adata_pb_L3.obs.columns if adata_pb_L3.obs[x].dtype == 'object']

['batch_id',
 'hto_barcode',
 'hto_category',
 'pbmc_sample_id',
 'pool_id',
 'sample.sampleKitGuid',
 'cohort.cohortGuid',
 'subject.subjectGuid',
 'subject.biologicalSex',
 'subject.race',
 'subject.ethnicity',
 'subject.birthYear',
 'sample.visitName',
 'sample.drawDate',
 'sample.diseaseStatesRecordedAtVisit',
 'sample.daysSinceFirstVisit',
 'file.id',
 'subset_grp',
 'predicted_doublet',
 'AIFI_L3',
 'doublets_manual',
 'AIFI_L3_new',
 'Status_Xsec',
 'Status_Long',
 'anti_ccp3_finalCombined',
 'days_to_conversion',
 'BMI',
 'CMV_Status_Subj',
 'age_conv',
 'bmi_conv',
 'file.batchID',
 'status',
 'total_b_counts']

In [132]:
adata_pb_L3

AnnData object with n_obs × n_vars = 813 × 26049
    obs: 'batch_id', 'hto_barcode', 'hto_category', 'pbmc_sample_id', 'pool_id', 'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid', 'subject.biologicalSex', 'subject.race', 'subject.ethnicity', 'subject.birthYear', 'sample.visitName', 'sample.drawDate', 'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit', 'file.id', 'subset_grp', 'predicted_doublet', 'AIFI_L3', 'doublets_manual', 'AIFI_L3_new', 'Status_Xsec', 'Status_Long', 'anti_ccp3_finalCombined', 'days_to_conversion', 'BMI', 'CMV_Status_Subj', 'age_conv', 'bmi_conv', 'file.batchID', 'status', 'total_b_counts', 'psbulk_n_cells', 'psbulk_counts'
    var: 'mean', 'std'
    layers: 'psbulk_props'

In [133]:
adata_pb_L3.obs.columns

Index(['batch_id', 'hto_barcode', 'hto_category', 'pbmc_sample_id', 'pool_id',
       'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid',
       'subject.biologicalSex', 'subject.race', 'subject.ethnicity',
       'subject.birthYear', 'sample.visitName', 'sample.drawDate',
       'sample.diseaseStatesRecordedAtVisit', 'sample.daysSinceFirstVisit',
       'file.id', 'subset_grp', 'predicted_doublet', 'AIFI_L3',
       'doublets_manual', 'AIFI_L3_new', 'Status_Xsec', 'Status_Long',
       'anti_ccp3_finalCombined', 'days_to_conversion', 'BMI',
       'CMV_Status_Subj', 'age_conv', 'bmi_conv', 'file.batchID', 'status',
       'total_b_counts', 'psbulk_n_cells', 'psbulk_counts'],
      dtype='object')

In [134]:
adata_pb_L3.layers['counts'] = adata_pb_L3.X.copy()

In [135]:
adata_pb_L3.obs.age_conv = adata_pb_L3.obs.age_conv.astype('float64')

In [136]:
adata_pb_L3.obs['predicted_doublet'] = adata_pb_L3.obs['predicted_doublet'].astype('bool')

In [137]:
adata_pb_L3.obs['sample.daysSinceFirstVisit'] = adata_pb_L3.obs['sample.daysSinceFirstVisit'].astype('int64')

In [138]:
adata_pb_L3.obs.total_b_counts = adata_pb_L3.obs.total_b_counts.astype('int64')

In [139]:
adata_pb_L3.obs.anti_ccp3_finalCombined = adata_pb_L3.obs.anti_ccp3_finalCombined.astype('float64')

In [140]:
adata_pb_L3.obs['days_to_conversion'] = adata_pb_L3.obs['days_to_conversion'].astype('float64')

In [141]:
adata_pb_L3.obs['BMI'] = adata_pb_L3.obs['BMI'].astype('float64')

In [142]:
adata_pb_L3.obs['bmi_conv'] = adata_pb_L3.obs['bmi_conv'].astype('float64')

In [143]:
adata_pb_L3.obs['subject.birthYear'] = adata_pb_L3.obs['subject.birthYear'].astype('int64')

In [144]:
adata_pb_L3.obs['subject.birthYear'] = adata_pb_L3.obs['subject.birthYear'].astype('int64')

### Save

In [145]:
out_ad_fp = os.path.join(outdir,'ALTRA_certPro_scRNA_141_samples_combined_adata_B_L3_pseudobulk_20cell.h5ad')
adata_pb_L3.write_h5ad(out_ad_fp)

## Upload Results

In [155]:
{x['name'] : x['id'] for x in hp.get_study_spaces()}

{'In vitro T cell perturbations caused by VRd therapy': '40df6403-29f0-4b45-ab7d-f46d420c422e',
 'TEA-seq: Simultaneous trimodal single-cell measurement of transcripts, epitopes, and chromatin accessibility': '350ce228-b130-4524-914d-7dcb7befe6a5',
 'Custom Dash App Plot Development': 'd72fb3ce-3a6d-41b5-9bb1-5fd3159312da',
 'Pre-RA Longitudinal Analysis': '652f8bc9-e0bd-45f6-82a5-b9ff524c0c70',
 'HISE NOVA Dev': '119f0b9e-6717-4892-a163-a0a986db8a37',
 'preRA cross-sectional + earlyRA + Longitudinal analysis': '223de760-9624-45bd-aefe-ca24c75b1800',
 'IH&A Analysis': 'de025812-5e73-4b3c-9c3b-6d0eac412f2a',
 'IH&A Data Apps (for testing)': 'c1e0e3d8-9914-43b4-a932-1776c6197908'}

In [157]:
ssid = '223de760-9624-45bd-aefe-ca24c75b1800'

In [158]:
fid_in

['609f7543-d4d5-41e9-a3d2-8e50c3e7c61d',
 '89859cec-8d36-47fd-a165-f0f34ae90284']

In [160]:
# Cell level isotype calls, cutoff 0.5
hp.upload_files(
    files = ['./_output/certpro/adata-all-b_isotypes_cutoff0.5.csv'],
    study_space_id = ssid,
    title = 'scRNA B Cell Isotype Analysis Output',
    input_file_ids = fid_in,
    destination = 'scRNA_BCellIsotype/v1'
)

Cannot determine the current notebook.
1) /home/jupyter/ra-longitudinal/scRNA/Aim3_switched-followup/01_py_AssignIsotype.ipynb
2) /home/jupyter/ra-longitudinal/scRNA/Aim3_switched-followup/02_R_SwitchedFrequencyAnalysis.ipynb
3) /home/jupyter/ra-longitudinal/scRNA/Aim3_switched-followup/03_R_IGHExpressionAnalysis_byIsotype.ipynb
Please select (1-3) 


 1


you are trying to upload file_ids... ['./_output/certpro/adata-all-b_isotypes_cutoff0.5.csv']. Do you truly want to proceed?


(y/n) y


{'trace_id': '9022459a-13ee-4126-9830-ff85f04e7d66',
 'files': ['./_output/certpro/adata-all-b_isotypes_cutoff0.5.csv']}

In [161]:
# Cell level isotype calls, cutoff 2
hp.upload_files(
    files = ['./_output/certpro/adata-all-b_isotypes_cutoff2.csv'],
    study_space_id = ssid,
    title = 'scRNA B Cell Isotype Counts, cell level table, cutoff 2',
    input_file_ids = fid_in,
    destination = 'scRNA_BCellIsotype/v1'
)

you are trying to upload file_ids... ['./_output/certpro/adata-all-b_isotypes_cutoff2.csv']. Do you truly want to proceed?


(y/n) y


{'trace_id': '0afb172c-3645-4bdf-b5de-ec9c2eba02aa',
 'files': ['./_output/certpro/adata-all-b_isotypes_cutoff2.csv']}

In [163]:
# All Cell Isotype Thresholds
hp.upload_files(
    files = ["./_output/certpro/check_igh_normcounts_alltypes.csv"],
    study_space_id = ssid,
    title = 'scRNA All Celltype IgH Isotype Threshold Check',
    input_file_ids = ['609f7543-d4d5-41e9-a3d2-8e50c3e7c61d'],
    destination = 'scRNA_BCellIsotype/v1'
)

you are trying to upload file_ids... ['./_output/certpro/check_igh_normcounts_alltypes.csv']. Do you truly want to proceed?


(y/n) y


{'trace_id': '98269fd8-fee7-4efc-a4c0-ad9e7fbb4785',
 'files': ['./_output/certpro/check_igh_normcounts_alltypes.csv']}

In [164]:
# Pseudobulk by Isotype B cell object
hp.upload_files(
    files = ['./_output/certpro/ALTRA_certPro_scRNA_141_samples_combined_adata_B_isotype_pseudobulk_20cell.h5ad'],
    study_space_id = ssid,
    title = 'scRNA B Cell Isotype Pseudobulk H5ad',
    input_file_ids = fid_in,
    destination = 'scRNA_BCellIsotype/v1'
)

you are trying to upload file_ids... ['./_output/certpro/ALTRA_certPro_scRNA_141_samples_combined_adata_B_isotype_pseudobulk_20cell.h5ad']. Do you truly want to proceed?


(y/n) y


{'trace_id': 'e149353f-abef-42b7-93db-55d9c73b3c09',
 'files': ['./_output/certpro/ALTRA_certPro_scRNA_141_samples_combined_adata_B_isotype_pseudobulk_20cell.h5ad']}

## Session Info

In [165]:
import session_info

In [166]:
session_info.show()